In [1]:
# Development playground for util functions
from types import *
from dataloader import *
from util import *
from graphs import *
from agreement import *
from scoring import *

data = load_data('../annotated', preprocess=True)

Loading files: ['../annotated/batch_1_anton.json', '../annotated/batch_1_ayush.json', '../annotated/batch_1_kelly.json', '../annotated/batch_2_vinayak.json', '../annotated/batch_2_vishnesh.json', '../annotated/batch_3_anton.json', '../annotated/batch_3_ayush.json', '../annotated/batch_3_kelly.json', '../annotated/batch_4_rachel.json', '../annotated/batch_4_vinayak.json', '../annotated/batch_4_vishnesh.json', '../annotated/batch_5_anton.json', '../annotated/batch_5_ayush.json']

Found users: {'rachel', 'vinayak', 'kelly', 'ayush', 'vishnesh', 'anton'}

anton - Batch 5, HIT 38 (ID 177) has 2 deletion edits but 1 annotations. Likely a missing annotation. Skipping edit type...
Couldn't process grammar for substitution: ['positive', '', '', '']. Assuming 'no'...
Couldn't process positive rating for substitution: ['positive', '', '', '']. Assuming 'somewhat'...
Couldn't process grammar for deletion: ['good', 'no', '']. Assuming 'no'...
Couldn't process grammar for substitution: ['positive', 

In [2]:
# For each family of edits
    # For each model type
        # For quality / error types
            # Graph a sum bar of number of edits

In [ ]:
[sent for sent in dat a]

In [5]:
data[0]['processed_annotations']

[{'edit_type': 'deletion',
  'id': 0,
  'information_impact': <Information.LESS: 'Less Information'>,
  'type': <Quality.QUALITY: 'No Error'>,
  'family': <Family.CONTENT: 'Content'>,
  'grammar_error': True,
  'error_type': None,
  'rating': 2,
  'size': 0.09278350515463918,
  'score': -2.9932336588376685}]

In [4]:
data[0]['edits']

[{'type': 'deletion',
  'id': 0,
  'original_span': [(11, 20)],
  'simplified_span': None,
  'annotation': ['good', 'yes', 'no']}]